### **Importing Necessary Libraries**

In [2]:
import pandas as pd
from datetime import datetime

# Collecting US FRED data with below libraries

import yfinance as yf
import pandas_datareader.data as web

In [3]:
# Defining Timeline

start = datetime(1980, 1, 1)
end = datetime(2019, 11, 26)

# Macro Signals

> FRED - Federal Reserve Economic Data

## **US Consumer Price Index(CPI)**

> Fetching Inflation Rate
> > High inflation → lower real returns, often higher interest rates.

> > CPIAUCSL - Consumer Price Index for All Urban Consumers, All Items in U.S. City Average.
> > > Key economic indicator from the U.S. Bureau of Labor Statistics

## **US FEDERAL FUNDS EFFECTIVE RATE(FEDFUNDS)**

> Fetching Interest Rates
> > Affect cost of borrowing → impact on corporate profits & valuations.

> > FEDFUNDS - federal funds rate is the interest rate at which depository institutions trade federal funds at Federal Reserve Banks with each other overnight

## **US Gross Domestic Product**

> GDPC1 - US Overall GDP

In [5]:
indicators = ["CPIAUCSL", "FEDFUNDS", "GDPC1"]
custom_col = ["CPI", "FED_FUNDS", "GDP"]

macros_df = pd.DataFrame()
for indicator, col in zip(indicators, custom_col):
    macros = web.DataReader(indicator, "fred", start, end)
    macros_df[col] = macros

In [6]:
macros_df.head()

,CPI,FED_FUNDS,GDP
DATE,,,
1980-01-01,78.0,13.82,7341.557
1980-02-01,79.0,14.13,NaN
1980-03-01,80.1,17.19,NaN
1980-04-01,80.9,17.61,7190.289
1980-05-01,81.7,10.98,NaN


In [7]:
macros_df.tail()

,CPI,FED_FUNDS,GDP
DATE,,,
2019-07-01,255.802,2.40,20843.322
2019-08-01,256.036,2.13,NaN
2019-09-01,256.430,2.04,NaN
2019-10-01,257.155,1.83,20985.448
2019-11-01,257.879,1.55,NaN


In [8]:
macros_df.shape

(479, 3)

In [9]:
macros_df.isnull().sum()

CPI            0
FED_FUNDS      0
GDP          319
dtype: int64

In [10]:
macros_df['GDP'] = macros_df['GDP'].ffill()

In [11]:
# Resampling Monthly to Daily data

macros_df = macros_df.resample('D').ffill()

In [12]:
macros_df.shape

(14550, 3)

In [13]:
# Renaming date column

macros_df.index.name = "Date"

In [14]:
macros_df

,CPI,FED_FUNDS,GDP
Date,,,
1980-01-01,78.000,13.82,7341.557
1980-01-02,78.000,13.82,7341.557
1980-01-03,78.000,13.82,7341.557
1980-01-04,78.000,13.82,7341.557
1980-01-05,78.000,13.82,7341.557
...,...,...,...
2019-10-28,257.155,1.83,20985.448
2019-10-29,257.155,1.83,20985.448
2019-10-30,257.155,1.83,20985.448


# Industry Trends

## **NASDAQ100**

> NASDAQ100 - Nasdaq‑100 Index (^NDX) is a stock index made up of the 100 largest non‑financial companies listed on the Nasdaq stock exchange
> > Tracks big tech peers — when it rises, Apple usually benefits.

## **NASDAQ Composite**

> NASDAQ Composite - ^IXIC is a stock market index that includes almost all stocks listed on the Nasdaq stock exchange.
> > Signals broad tech participation — confirms strength behind Apple’s moves.

## **Standard and Poor's 500**

> S&P 500 - ^GPSC is a Stock market index tracking the stock performance of 500 leading companies listed on stock exchanges in the United States.
> > Shows Apple’s performance in the context of the overall U.S. market.

## **PHLX Semiconductor Index**
>  PHLX Semiconductor Sector - SOX is a capitalization-weighted index comprising the 30 largest U.S.-traded companies primarily involved in the design, distribution, manufacture, and sale of semiconductors
> > Leads Apple’s cycle by reflecting chip demand and supply chain health.

In [16]:
start = datetime(1980, 1, 1) # since NASDAQ introduces in 1985

In [17]:
custom_col = ["NASDAQ100", "SOX"]
tickers = ["^NDX", "^SOX"]

trend_df = pd.DataFrame()

for t, col in zip(tickers, custom_col):    
    try:
        df = yf.download(t, start=start, end=end, auto_adjust=False)
        trend_df[col] = df["Adj Close"]
    except Exception as e:
        print(f"Failed to download {ticker}: {e}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [18]:
trend_df.head()

,NASDAQ100,SOX
Date,,
1985-10-01,112.139999,NaN
1985-10-02,110.824997,NaN
1985-10-03,110.870003,NaN
1985-10-04,110.074997,NaN
1985-10-07,108.199997,NaN


In [19]:
trend_df.tail()

,NASDAQ100,SOX
Date,,
2019-11-19,8338.740234,1730.500000
2019-11-20,8283.750000,1710.140015
2019-11-21,8265.620117,1690.829956
2019-11-22,8272.049805,1690.790039
2019-11-25,8371.929688,1731.949951


In [20]:
trend_df.isnull().sum()

NASDAQ100       0
SOX          2172
dtype: int64

In [21]:
comp_df = yf.download("^IXIC", start=start, end=end, auto_adjust=False)["Close"]

[*********************100%***********************]  1 of 1 completed


In [22]:
comp_df.head()

Ticker,^IXIC
Date,
1980-01-02,148.169998
1980-01-03,145.970001
1980-01-04,148.020004
1980-01-07,148.619995
1980-01-08,150.679993


In [23]:
sp_df = yf.download("^GSPC", start=start, end=end, auto_adjust=False)["Close"]

[*********************100%***********************]  1 of 1 completed


In [24]:
sp_df.head()

Ticker,^GSPC
Date,
1980-01-02,105.760002
1980-01-03,105.220001
1980-01-04,106.519997
1980-01-07,106.809998
1980-01-08,108.949997


In [25]:
market_trend_df = comp_df.merge(trend_df, on='Date', how='outer').merge(sp_df, on='Date', how='outer')

In [26]:
market_trend_df.head()

,^IXIC,NASDAQ100,SOX,^GSPC
Date,,,,
1980-01-02,148.169998,NaN,NaN,105.760002
1980-01-03,145.970001,NaN,NaN,105.220001
1980-01-04,148.020004,NaN,NaN,106.519997
1980-01-07,148.619995,NaN,NaN,106.809998
1980-01-08,150.679993,NaN,NaN,108.949997


In [27]:
market_trend_df.rename(columns={"^IXIC" : "NASDAQ_Comp_Close",
                                "NASDAQ100" : "NASDAQ100_Adj_Close",
                                "SOX" : "SOX_Adj_Close",
                                "^GSPC" :"S&P500_Close"}, inplace=True)

In [28]:
market_trend_df.tail()

,NASDAQ_Comp_Close,NASDAQ100_Adj_Close,SOX_Adj_Close,S&P500_Close
Date,,,,
2019-11-19,8570.660156,8338.740234,1730.500000,3120.179932
2019-11-20,8526.730469,8283.750000,1710.140015,3108.459961
2019-11-21,8506.209961,8265.620117,1690.829956,3103.540039
2019-11-22,8519.879883,8272.049805,1690.790039,3110.290039
2019-11-25,8632.490234,8371.929688,1731.949951,3133.639893


In [29]:
market_trend_df.isnull().sum()

NASDAQ_Comp_Close         0
NASDAQ100_Adj_Close    1453
SOX_Adj_Close          3625
S&P500_Close              0
dtype: int64

In [30]:
market_trend_df = market_trend_df.fillna(0)

# Volatility Trends

## VIX

> VIX Index is used as a barometer for market uncertainty
> Popular name for the Chicago Board Options Exchange's CBOE Volatility Index, a popular measure of the stock market's expectation of volatility based on S&P 500 index options

In [90]:
vix = yf.download('^VIX', start=start, end=end, auto_adjust=False)["Adj Close"]

[*********************100%***********************]  1 of 1 completed


In [94]:
vix.rename(columns={"^VIX" : "VIX_Adj_Close"}, inplace=True)

In [96]:
vix.head()

Ticker,VIX_Adj_Close
Date,
1990-01-02,17.240000
1990-01-03,18.190001
1990-01-04,19.219999
1990-01-05,20.110001
1990-01-08,20.260000


## **Merging Macro Indicators**

In [98]:
macroeco_df = macros_df.merge(market_trend_df, on='Date', how='outer').merge(vix, on='Date', how='outer')

In [100]:
macroeco_df.head(50)

,CPI,FED_FUNDS,GDP,NASDAQ_Comp_Close,NASDAQ100_Adj_Close,SOX_Adj_Close,S&P500_Close,VIX_Adj_Close
Date,,,,,,,,
1980-01-01,78.0,13.82,7341.557,NaN,NaN,NaN,NaN,NaN
1980-01-02,78.0,13.82,7341.557,148.169998,0.0,0.0,105.760002,NaN
1980-01-03,78.0,13.82,7341.557,145.970001,0.0,0.0,105.220001,NaN
1980-01-04,78.0,13.82,7341.557,148.020004,0.0,0.0,106.519997,NaN
1980-01-05,78.0,13.82,7341.557,NaN,NaN,NaN,NaN,NaN
1980-01-06,78.0,13.82,7341.557,NaN,NaN,NaN,NaN,NaN
1980-01-07,78.0,13.82,7341.557,148.619995,0.0,0.0,106.809998,NaN
1980-01-08,78.0,13.82,7341.557,150.679993,0.0,0.0,108.949997,NaN
1980-01-09,78.0,13.82,7341.557,151.600006,0.0,0.0,109.050003,NaN


In [102]:
macroeco_df.isnull().sum()

CPI                      16
FED_FUNDS                16
GDP                      16
NASDAQ_Comp_Close      4503
NASDAQ100_Adj_Close    4503
SOX_Adj_Close          4503
S&P500_Close           4503
VIX_Adj_Close          7031
dtype: int64

In [104]:
macroeco_df.shape

(14566, 8)

In [106]:
macroeco_df = macroeco_df.ffill().fillna(0)

In [108]:
macroeco_df.isnull().sum()

CPI                    0
FED_FUNDS              0
GDP                    0
NASDAQ_Comp_Close      0
NASDAQ100_Adj_Close    0
SOX_Adj_Close          0
S&P500_Close           0
VIX_Adj_Close          0
dtype: int64

In [110]:
macroeco_df.head(50)

,CPI,FED_FUNDS,GDP,NASDAQ_Comp_Close,NASDAQ100_Adj_Close,SOX_Adj_Close,S&P500_Close,VIX_Adj_Close
Date,,,,,,,,
1980-01-01,78.0,13.82,7341.557,0.000000,0.0,0.0,0.000000,0.0
1980-01-02,78.0,13.82,7341.557,148.169998,0.0,0.0,105.760002,0.0
1980-01-03,78.0,13.82,7341.557,145.970001,0.0,0.0,105.220001,0.0
1980-01-04,78.0,13.82,7341.557,148.020004,0.0,0.0,106.519997,0.0
1980-01-05,78.0,13.82,7341.557,148.020004,0.0,0.0,106.519997,0.0
1980-01-06,78.0,13.82,7341.557,148.020004,0.0,0.0,106.519997,0.0
1980-01-07,78.0,13.82,7341.557,148.619995,0.0,0.0,106.809998,0.0
1980-01-08,78.0,13.82,7341.557,150.679993,0.0,0.0,108.949997,0.0
1980-01-09,78.0,13.82,7341.557,151.600006,0.0,0.0,109.050003,0.0


In [112]:
macroeco_df.to_csv("macroeconomic_data_1980_to_2019.csv")